## Install Libraries


In [41]:
%pip install ray[tune]
# %pip install bayesian-optimization
#!pip install git+https://github.com/yfw/starter-code
%pip install ray[tune] bayesian-optimization==1.2.0
%pip install hyperopt
%pip install scikit-optimize
# %pip install torch torchvision torchaudio # Uncomment this to install PyTorch 2.0+ on ilab (required version to run this program)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Libraries


In [42]:
from filelock import FileLock
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from torch.utils.data import random_split



import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from collections import OrderedDict

from ray.tune.schedulers import ASHAScheduler

from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.bayesopt import BayesOptSearch

from hyperopt import hp
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.search.skopt import SkOptSearch

## Data loaders


In [43]:
def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    with FileLock(os.path.expanduser("~/.data.lock")):
        trainset = torchvision.datasets.CIFAR10(
            root=data_dir, train=True, download=True, transform=transform
        )

        testset = torchvision.datasets.CIFAR10(
            root=data_dir, train=False, download=True, transform=transform
        )

    return trainset, testset


## Configurable neural network


In [44]:
def flatten(x, start_dim=1, end_dim=-1):
    return x.flatten(start_dim=start_dim, end_dim=end_dim)


In [45]:
# Will need to modify the parameters and the network depending on what you are experimenting with

C, H, W = 3, 32, 32
num_classes = 10

channel_1 = 32
channel_2 = 64
channel_3 = 128
pool_kernel_size = 2

kernel_size_1 = 3
kernel_size_2 = 5
kernel_size_3 = 7

pad_size_1 = 2
pad_size_2 = 3
pad_size_3 = 3

fc_count_1 = 1024

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, channel_1, kernel_size_1, padding=pad_size_1)
        self.norm1 = nn.BatchNorm2d(channel_1)
        self.pool1 = nn.MaxPool2d(pool_kernel_size)
        self.conv2 = nn.Conv2d(channel_1, channel_2, kernel_size_2, padding=pad_size_2)
        self.norm2 = nn.BatchNorm2d(channel_2)
        self.pool2 = nn.MaxPool2d(pool_kernel_size)
        self.conv3 = nn.Conv2d(channel_2, channel_3, kernel_size_3, padding=pad_size_3)
        self.norm3 = nn.BatchNorm2d(channel_3)
        self.pool3 = nn.MaxPool2d(pool_kernel_size)        

        H_after_conv1 = int(H + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        W_after_conv1 = int(W + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        H_after_pool1 = int((H_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool1 = int((W_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        H_after_conv2 = int(H_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        W_after_conv2 = int(W_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        H_after_pool2 = int((H_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool2 = int((W_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)  
        H_after_conv3 = int(H_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        W_after_conv3 = int(W_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        H_after_pool3 = int((H_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool3 = int((W_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)

        self.fc1 = nn.Linear(H_after_pool3*W_after_pool3*channel_3, fc_count_1)
        self.fc2 = nn.Linear(fc_count_1, fc_count_1)
        self.fc3 = nn.Linear(fc_count_1, 10)


    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = F.relu(x)
        x = self.pool3(x)
        x = flatten(x)
        x = self.fc1(x)
        #x = F.relu(x)
        x = self.fc2(x)
        #x = F.relu(x)
        x = self.fc3(x)

        return x



## The train function


In [46]:
def train_cifar(config):
    net = Net()  # Will need to modify the parameters depending on what you are experimenting with

    learning_rate = config['learning_rate']
    momentum = config['momentum']
    weight_decay = 0.001

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.SGD(net.parameters(), lr=learning_rate, 
                weight_decay=weight_decay,
                momentum=momentum, nesterov=True)    




    # Do not modify any code below!
    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state = torch.load(
                os.path.join(loaded_checkpoint_dir, "checkpoint.pt")
            )
            net.load_state_dict(model_state)
            optimizer.load_state_dict(optimizer_state)

    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=64, shuffle=True, num_workers=8
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=64, shuffle=True, num_workers=8
    )

    for epoch in range(10):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:
                print(
                    f"[{epoch + 1}, {i + 1:>5} loss: {running_loss / epoch_steps:.3f}]"
                )
                running_loss = 0.0

        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        os.makedirs("my_model", exist_ok=True)
        torch.save((net.state_dict(), optimizer.state_dict()), "my_model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("my_model")
        session.report(
            {"loss": (val_loss / val_steps), "accuracy": correct / total},
            checkpoint=checkpoint,
        )
    print("Finished Training")


## Test set accuracy


In [47]:
def test_best_model(best_result):
    best_trained_model = Net(
     )  # Will need to modify the parameters depending on what you are experimenting with

    # Do not modify any code below!
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(
        best_result.checkpoint.to_directory(), "checkpoint.pt"
    )

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Best trial test set accuracy: {correct / total}")


## Configuring the search space


In [48]:
# You will be experimenting with the hyperparameters here
# Use tune.grid_search to specify what values you want to experiment with a hyperparameter
config = { 
          "learning_rate": tune.uniform(1e-3, 1e-1),
          "momentum": tune.uniform(0.1, 0.9),
          }



In [49]:
# Do not modify any code below!
def main(config):

  #hyperopt_search = HyperOptSearch(config, metric="accuracy", mode="max")
  #algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0})
  #algo = ConcurrencyLimiter(algo, max_concurrent=4)

  current_best_params = [
    {
        "learning_rate": 0.053306,
        "momentum": 0.570568,
    },
    {
        "learning_rate": 0.028210,
        "momentum": 0.338465,
    },
    {
        "learning_rate": 0.013342,
        "momentum": 0.605203,
    }
  ]

  skopt_search = SkOptSearch(
    metric="accuracy",
    mode="max",
    points_to_evaluate=current_best_params)

  tuner = tune.Tuner(
    tune.with_resources(
      tune.with_parameters(train_cifar),
      resources={"gpu": 1},
    ),
    # tune_config=tune.TuneConfig(
    #   metric="accuracy",
    #   mode="max",
    #   search_alg=algo,
    # ),
    tune_config=tune.TuneConfig(
      num_samples = 30,
      search_alg=skopt_search,
    ),
      param_space=config,
    )
  results = tuner.fit()

  best_result = results.get_best_result("accuracy", "max")

  print(f"Best trial config: {best_result.config}")
  print(f"Best trial final validation loss: {best_result.metrics['loss']}")
  print(f"Best trial final validation accuracy: {best_result.metrics['accuracy']}")

  test_best_model(best_result)

  return results.get_dataframe("accuracy", "max")


results_df = main(config)


(train_cifar pid=21616) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-13_07-29-08/train_cifar_683fcf7c_1_learning_rate=0.0533,momentum=0.5706_2023-04-13_07-29-08/data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:05<00:00, 28645568.23it/s]


(train_cifar pid=21616) Extracting /root/ray_results/train_cifar_2023-04-13_07-29-08/train_cifar_683fcf7c_1_learning_rate=0.0533,momentum=0.5706_2023-04-13_07-29-08/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-13_07-29-08/train_cifar_683fcf7c_1_learning_rate=0.0533,momentum=0.5706_2023-04-13_07-29-08/data
(train_cifar pid=21616) Files already downloaded and verified


Trial name,accuracy,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_cifar_055f429e,0.7594,2023-04-13_07-34-15,True,,5a843aadb3794948bfd80fe491aa1888,"5_learning_rate=0.0347,momentum=0.5038",ed1eb21d0f80,10,0.842427,172.28.0.12,21616,True,60.0488,4.96094,60.0488,1681371255,0,,10,055f429e,0.00421739
train_cifar_05fba8d5,0.6613,2023-04-13_07-57-42,True,,5a843aadb3794948bfd80fe491aa1888,"28_learning_rate=0.0010,momentum=0.1000",ed1eb21d0f80,10,0.98104,172.28.0.12,21616,True,61.6278,4.98777,61.6278,1681372662,0,,10,05fba8d5,0.00421739
train_cifar_21a435ff,0.7591,2023-04-13_07-49-24,True,,5a843aadb3794948bfd80fe491aa1888,"20_learning_rate=0.0152,momentum=0.9000",ed1eb21d0f80,10,0.829659,172.28.0.12,21616,True,59.3178,4.96435,59.3178,1681372164,0,,10,21a435ff,0.00421739
train_cifar_22b5e68d,0.6661,2023-04-13_07-56-40,True,,5a843aadb3794948bfd80fe491aa1888,"27_learning_rate=0.0010,momentum=0.1000",ed1eb21d0f80,10,0.957105,172.28.0.12,21616,True,73.4537,4.89127,73.4537,1681372600,0,,10,22b5e68d,0.00421739
train_cifar_259d1be5,0.1011,2023-04-13_07-33-15,True,,5a843aadb3794948bfd80fe491aa1888,"4_learning_rate=0.0867,momentum=0.6246",ed1eb21d0f80,10,nan,172.28.0.12,21616,True,61.4143,5.01028,61.4143,1681371195,0,,10,259d1be5,0.00421739
train_cifar_26ad1bb6,0.7448,2023-04-13_07-48-25,True,,5a843aadb3794948bfd80fe491aa1888,"19_learning_rate=0.0376,momentum=0.9000",ed1eb21d0f80,10,0.76854,172.28.0.12,21616,True,59.0038,4.93554,59.0038,1681372105,0,,10,26ad1bb6,0.00421739
train_cifar_2c66be32,0.7622,2023-04-13_07-46-25,True,,5a843aadb3794948bfd80fe491aa1888,"17_learning_rate=0.0371,momentum=0.1000",ed1eb21d0f80,10,0.805857,172.28.0.12,21616,True,60.0794,5.0006,60.0794,1681371985,0,,10,2c66be32,0.00421739
train_cifar_32a9852d,0.7589,2023-04-13_07-43-20,True,,5a843aadb3794948bfd80fe491aa1888,"14_learning_rate=0.0186,momentum=0.1000",ed1eb21d0f80,10,0.778791,172.28.0.12,21616,True,62.2223,4.98443,62.2223,1681371800,0,,10,32a9852d,0.00421739
train_cifar_3bd26127,0.7501,2023-04-13_07-50-24,True,,5a843aadb3794948bfd80fe491aa1888,"21_learning_rate=0.0152,momentum=0.9000",ed1eb21d0f80,10,0.891411,172.28.0.12,21616,True,59.1667,5.04606,59.1667,1681372224,0,,10,3bd26127,0.00421739
train_cifar_4613e3c4,0.7371,2023-04-13_07-41-16,True,,5a843aadb3794948bfd80fe491aa1888,"12_learning_rate=0.0479,momentum=0.7562",ed1eb21d0f80,10,0.841592,172.28.0.12,21616,True,59.0522,4.8759,59.0522,1681371676,0,,10,4613e3c4,0.00421739


(train_cifar pid=21616) Finished Training
(train_cifar pid=21616) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-13_07-29-08/train_cifar_58540319_2_learning_rate=0.0282,momentum=0.3385_2023-04-13_07-30-14/data/cifar-10-python.tar.gz


...

(train_cifar pid=21616) Extracting /root/ray_results/train_cifar_2023-04-13_07-29-08/train_cifar_49d6b556_29_learning_rate=0.0077,momentum=0.9000_2023-04-13_07-57-42/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-13_07-29-08/train_cifar_49d6b556_29_learning_rate=0.0077,momentum=0.9000_2023-04-13_07-57-42/data
(train_cifar pid=21616) Files already downloaded and verified
(train_cifar pid=21616) Finished Training
(train_cifar pid=21616) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-13_07-29-08/train_cifar_f6e5aa1b_30_learning_rate=0.0072,momentum=0.9000_2023-04-13_07-58-43/data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:05<00:00, 28902219.61it/s]


(train_cifar pid=21616) Extracting /root/ray_results/train_cifar_2023-04-13_07-29-08/train_cifar_f6e5aa1b_30_learning_rate=0.0072,momentum=0.9000_2023-04-13_07-58-43/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-13_07-29-08/train_cifar_f6e5aa1b_30_learning_rate=0.0072,momentum=0.9000_2023-04-13_07-58-43/data
(train_cifar pid=21616) Files already downloaded and verified
(train_cifar pid=21616) Finished Training


2023-04-13 07:59:43,849	INFO tune.py:798 -- Total run time: 1835.51 seconds (1835.47 seconds for the tuning loop).


Best trial config: {'learning_rate': 0.007197361830645431, 'momentum': 0.9}
Best trial final validation loss: 0.886002612341741
Best trial final validation accuracy: 0.7669
Files already downloaded and verified
Files already downloaded and verified
Best trial test set accuracy: 0.7372


In [50]:
final_df = results_df.drop(
    columns=[
        "should_checkpoint",
        "timesteps_total",
        "episodes_total",
        "training_iteration",
        "trial_id",
        "experiment_id",
        "date",
        "timestamp",
        "pid",
        "hostname",
        "node_ip",
        "time_since_restore",
        "timesteps_since_restore",
        "iterations_since_restore",
        "warmup_time",
        "logdir",
    ]
)
final_df.sort_values(by=['accuracy'], ascending=False).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/learning_rate,config/momentum
19,0.665818,0.7707,4.944207,False,34.603192,0.015221,0.900000
17,0.709110,0.7676,5.068652,False,44.615632,0.037195,0.100000
29,0.886003,0.7669,5.039817,False,59.404519,0.007197,0.900000
20,0.761133,0.7665,4.855726,False,49.241999,0.015193,0.900000
7,0.755334,0.7661,4.920707,False,57.501354,0.025539,0.210218
16,0.688952,0.7656,4.912491,False,40.131042,0.037067,0.100000
28,0.874712,0.7654,5.123604,False,60.627084,0.007662,0.900000
22,0.725938,0.7652,4.895066,False,50.017725,0.050633,0.100000
1,0.729516,0.7645,4.839552,False,44.469491,0.028210,0.338465
25,0.732522,0.7641,4.961905,False,44.074652,0.022682,0.900000


In [51]:
final_df.sort_values(by=['accuracy']).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/learning_rate,config/momentum
9,NaN,0.0986,14.474193,False,14.474193,0.096245,0.455211
8,NaN,0.1003,16.165922,False,16.165922,0.073345,0.759311
3,NaN,0.1011,16.965347,False,16.965347,0.086741,0.624554
27,0.981040,0.6613,4.987771,False,61.627751,0.001000,0.100000
26,0.957105,0.6661,4.891266,False,73.453667,0.001000,0.100000
6,0.768737,0.7405,4.977035,False,44.075492,0.008259,0.119263
18,0.768540,0.7448,4.935539,False,59.003820,0.037597,0.900000
12,0.859787,0.7461,4.955603,False,61.418235,0.001000,0.900000
11,0.766995,0.7497,5.116255,False,49.201387,0.047927,0.756176
14,0.745935,0.7569,4.997670,False,52.042447,0.019149,0.100000
